In [74]:
import pandas_datareader.data as web
import pandas as pd
import numpy as np
import datetime
from tqdm import tqdm
import plotly
import plotly.graph_objects as go

In [75]:
start = datetime.datetime(2018,10,4)
end =datetime.datetime(2022,10,4)

In [76]:
BTC = web.DataReader('BTC-USD', 'yahoo', start, end)
ETH = web.DataReader('ETH-USD', 'yahoo', start, end)
USDT = web.DataReader('USDT-USD', 'yahoo', start, end) 
BNB = web.DataReader('BNB-USD', 'yahoo', start, end)
print(BTC,ETH,USDT,BNB)

                    High           Low          Open         Close  \
Date                                                                 
2018-10-04   6603.310059   6497.910156   6497.910156   6576.689941   
2018-10-05   6623.620117   6557.410156   6574.149902   6622.479980   
2018-10-06   6628.540039   6577.799805   6622.450195   6588.310059   
2018-10-07   6641.490234   6557.040039   6590.680176   6602.950195   
2018-10-08   6675.060059   6576.040039   6600.189941   6652.229980   
...                  ...           ...           ...           ...   
2022-10-01  19471.154297  19231.082031  19431.105469  19312.095703   
2022-10-02  19370.308594  18970.621094  19311.849609  19044.107422   
2022-10-03  19653.542969  19025.226562  19044.068359  19623.580078   
2022-10-04  20380.343750  19523.839844  19623.583984  20336.843750   
2022-10-05  20343.748047  19801.800781  20335.900391  20160.716797   

                 Volume     Adj Close  
Date                                   
2018-10-0

In [77]:
daily_returns = pd.concat([BTC['Adj Close'], ETH['Adj Close'], USDT['Adj Close'], BNB['Adj Close']],axis=1)
daily_returns.columns = ['BTC', 'ETH', 'USDT', 'BNB'] 
daily_returns

,BTC,ETH,USDT,BNB
Date,,,,
2018-10-04,6576.689941,222.218002,0.996185,10.255600
2018-10-05,6622.479980,227.600998,0.995778,10.636200
2018-10-06,6588.310059,225.119995,0.997177,10.372100
2018-10-07,6602.950195,226.119003,0.996617,10.519000
2018-10-08,6652.229980,229.255005,0.995814,10.506600
...,...,...,...,...
2022-10-01,19312.095703,1311.644409,1.000043,282.871918
2022-10-02,19044.107422,1276.093506,1.000060,284.433105
2022-10-03,19623.580078,1323.439209,1.000089,287.031982


In [78]:
returns = daily_returns.shift(1)/daily_returns
logreturns = np.log(returns)*252
cov = daily_returns.cov()*252
logreturns

,BTC,ETH,USDT,BNB
Date,,,,
2018-10-04,NaN,NaN,NaN,NaN
2018-10-05,-1.748464,-6.031669,0.102973,-9.182723
2018-10-06,1.303607,2.762050,-0.353789,6.336231
2018-10-07,-0.559358,-1.115819,0.141556,-3.544041
2018-10-08,-1.873768,-3.470927,0.203123,0.297230
...,...,...,...,...
2022-10-01,1.557037,3.118838,0.001502,1.173535
2022-10-02,3.521420,6.924497,-0.004266,-1.386980
2022-10-03,-7.553492,-9.180447,-0.007329,-2.292079


In [79]:
cov

,BTC,ETH,USDT,BNB
BTC,8.211623e+10,5.520770e+09,-4620.015375,7.906955e+08
ETH,5.520770e+09,4.318443e+08,-312.848461,6.158315e+07
USDT,-4.620015e+03,-3.128485e+02,0.007228,-4.527289e+01
BNB,7.906955e+08,6.158315e+07,-45.272886,9.367809e+06


In [80]:
n_assets = 4 
n_portfolios = 1000
mean_variance_pairs = []
np.random.seed(75)
for i in range(n_portfolios):
    assets = np.random.choice(list(daily_returns.columns), n_assets, replace=False)
    weights = np.random.rand(n_assets)
    weights = weights/sum(weights)
portfolio_E_Variance = 0 
portfolio_E_Return = 0
for i in range(len(assets)):
    portfolio_E_Return += weights[i] * mus.loc[assets[i]]
    for j in range(len(assets)):
        portfolio_E_Variance += weights[i] * weights[j] * cov.loc[assets[i], assets[j]]
    
mean_variance_pairs.append([portfolio_E_Return, portfolio_E_Variance])

In [82]:
mean_variance_pairs = np.array(mean_variance_pairs)
risk_free_rate=0
fig = go.Figure()
fig.add_trace(go.Scatter(x=mean_variance_pairs[:,1]**0.5, y=mean_variance_pairs[:,0], 
                      marker=dict(color=(mean_variance_pairs[:,0]-risk_free_rate)/(mean_variance_pairs[:,1]**0.5), 
                                  showscale=True, 
                                  size=7,
                                  line=dict(width=1),
                                  colorscale="RdBu",
                                  colorbar=dict(title="Sharpe<br>Ratio")
                                 ), 
                      mode='markers'))
fig.update_layout(template='plotly_white',
                  xaxis=dict(title='Annualised Risk (Volatility)'),
                  yaxis=dict(title='Annualised Return'),
                  title='Sample of Random Portfolios',
                  width=900,
                  height=600)
fig.update_xaxes(range=[0.18, 0.32])
fig.update_yaxes(range=[0.02,0.27])
fig.update_layout(coloraxis_colorbar=dict(title="Sharpe Ratio"))